# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [56]:
# Dependencies and Setup
import pandas as pd
import matplotlib.pyplot as plt
import requests
import gmaps

# Import API key
from config import g_key


In [57]:
# Configure gmaps
gmaps.configure(api_key=g_key)


## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [58]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")

# Display sample data
vacation_df.head()


,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,2,Hilo,US,76.12,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
1,13,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
2,19,Oranjestad,AW,80.55,few clouds,12.5240,-70.0270,Divi Village Golf & Beach Resort
3,21,Hithadhoo,MV,81.84,overcast clouds,-0.6000,73.0833,Scoop Guest House
4,23,Kavieng,PG,85.10,broken clouds,-2.5744,150.7967,Nusa Island Retreat


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [59]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [60]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]


In [61]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [62]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[(vacation_df["City"] == "Pakenham")]
vacation_end = vacation_df.loc[(vacation_df["City"] == "Pakenham")]
vacation_stop1 = vacation_df.loc[(vacation_df["City"] == "Healesville")]
vacation_stop2 = vacation_df.loc[(vacation_df["City"] == "Kiama")]
vacation_stop3 = vacation_df.loc[(vacation_df["City"] == "Leeton")]

vacation_start

,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
133,580,Pakenham,AU,83.46,overcast clouds,-38.0667,145.4833,Pakenham Ranges


## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [63]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start[['Lat','Lng']].to_numpy()
start = start[0]
end   = vacation_end[['Lat','Lng']].to_numpy()
end   = end[0]
stop1 = vacation_stop1[['Lat','Lng']].to_numpy()
stop1 = stop1[0]
stop2 = vacation_stop2[['Lat','Lng']].to_numpy()
stop2 = stop2[0]
stop3 = vacation_stop3[['Lat','Lng']].to_numpy()
stop3 = stop3[0]

len(start)

2

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [64]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, 
# and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(start, end, waypoints = [stop1, stop2, stop3], travel_mode = 'DRIVING')

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig


Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [49]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df


,City_ID,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,580,Pakenham,AU,83.46,overcast clouds,-38.0667,145.4833,Pakenham Ranges
1,368,Healesville,AU,83.21,overcast clouds,-37.6500,145.5167,Tuck Inn Yarra Valley Accommodation Healesville
2,76,Kiama,AU,76.37,scattered clouds,-34.6833,150.8667,The Sebel Kiama Harbourside
3,290,Leeton,AU,81.90,overcast clouds,-34.5667,146.4000,Hydro Hotel


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [51]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]# YOUR CODE HERE

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]# YOUR CODE HERE



In [52]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Define a new figure object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig


Figure(layout=FigureLayout(height='420px'))